In [1]:
import torch
import argparse
import warnings
import numpy as np
import pandas as pd
from src.graph_func import graph_construction
from src.utils_func import mk_dir, adata_preprocess, load_ST_file
import anndata
from src.SEDR_train import SEDR_Train
from sklearn import metrics
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq
import os

warnings.filterwarnings('ignore')
torch.cuda.cudnn_enabled = False
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('===== Using device: ' + device)

===== Using device: cuda:0


In [2]:
def res_search_fixed_clus(adata, fixed_clus_count, increment=0.02):
    '''
        arg1(adata)[AnnData matrix]
        arg2(fixed_clus_count)[int]
        
        return:
            resolution[int]
    '''
    for res in sorted(list(np.arange(0.2, 2.5, increment)), reverse=True):
        sc.tl.leiden(adata, random_state=0, resolution=res)
        count_unique_leiden = len(pd.DataFrame(adata.obs['leiden']).leiden.unique())
        if count_unique_leiden == fixed_clus_count:
            break
    return res

In [3]:
# ################ Parameter setting
parser = argparse.ArgumentParser()
parser.add_argument('--k', type=int, default=10, help='parameter k in spatial graph')
parser.add_argument('--knn_distanceType', type=str, default='euclidean',
                    help='graph distance type: euclidean/cosine/correlation')
parser.add_argument('--epochs', type=int, default=200, help='Number of epochs to train.')
parser.add_argument('--cell_feat_dim', type=int, default=200, help='Dim of PCA')
parser.add_argument('--feat_hidden1', type=int, default=100, help='Dim of DNN hidden 1-layer.')
parser.add_argument('--feat_hidden2', type=int, default=20, help='Dim of DNN hidden 2-layer.')
parser.add_argument('--gcn_hidden1', type=int, default=32, help='Dim of GCN hidden 1-layer.')
parser.add_argument('--gcn_hidden2', type=int, default=8, help='Dim of GCN hidden 2-layer.')
parser.add_argument('--p_drop', type=float, default=0.2, help='Dropout rate.')
parser.add_argument('--using_dec', type=bool, default=True, help='Using DEC loss.')
parser.add_argument('--using_mask', type=bool, default=False, help='Using mask for multi-dataset.')
parser.add_argument('--feat_w', type=float, default=10, help='Weight of DNN loss.')
parser.add_argument('--gcn_w', type=float, default=0.1, help='Weight of GCN loss.')
parser.add_argument('--dec_kl_w', type=float, default=10, help='Weight of DEC loss.')
parser.add_argument('--gcn_lr', type=float, default=0.01, help='Initial GNN learning rate.')
parser.add_argument('--gcn_decay', type=float, default=0.01, help='Initial decay rate.')
parser.add_argument('--dec_cluster_n', type=int, default=10, help='DEC cluster number.')
parser.add_argument('--dec_interval', type=int, default=20, help='DEC interval nnumber.')
parser.add_argument('--dec_tol', type=float, default=0.00, help='DEC tol.')
# ______________ Eval clustering Setting ______________
parser.add_argument('--eval_resolution', type=int, default=1, help='Eval cluster number.')
parser.add_argument('--eval_graph_n', type=int, default=20, help='Eval graph kN tol.') 
parser.add_argument('-f')

params = parser.parse_args(args=[])
# args, unknown = parser.parse_known_args()
params.device = device

In [4]:
# ################ Path setting
# data_root = './data/DLPFC'
data_root = '/home/tengliu/Torch_pyG/Slide_seqV2_Comparison/SPGDL/data/Slide-seqV2'
save_root = '/home/tengliu/Paper2_Genome_Biology/10. Slide-seqV2_training/Benchmarking methods/Results'
data_name = 'Slide_seqV2'
n_clusters = 14

# adata_h5=sc.read('/home/tengliu/Torch_pyG/Slide_seqV2_Comparison/SEDR/Slide_seqV2_adataforSEDR.h5ad')
adata_h5 = sq.datasets.slideseqv2() # 4000高变基因
adata_h5.var_names_make_unique()
adata_h5= adata_h5[(adata_h5.obs["x"]>=1000) & (adata_h5.obs["x"]<=5600) & (adata_h5.obs["y"]<=5400) & (adata_h5.obs["y"]>=2600)]

adata_X = adata_preprocess(adata_h5, min_cells=5, pca_n_comps=params.cell_feat_dim)
graph_dict = graph_construction(adata_h5.obsm['spatial'], adata_h5.shape[0], params)
params.save_path = mk_dir(f'save_root/SEDR')

params.cell_num = adata_h5.shape[0]
print('==== Graph Construction Finished')

===== Preprocessing Data 
==== Graph Construction Finished


In [6]:
adata_h5.obs["cluster"]

TACTTTAGCGCAGT    CA1_CA2_CA3_Subiculum
TCGATATGGCACAA          DentatePyramids
ACGGATGTTCCGAT         Oligodendrocytes
TCTCATGGGTGGGA               Astrocytes
ACAGGGTTTATCGA    CA1_CA2_CA3_Subiculum
                          ...          
CTGGCTTTAGACCT    CA1_CA2_CA3_Subiculum
CGTCGGATGTAGGG                Microglia
CTTGATTAGGCAGG             Interneurons
GCGCTCAGTGCTCT             Interneurons
GAGATAACAGTTCC               Astrocytes
Name: cluster, Length: 26146, dtype: category
Categories (14, object): ['Astrocytes', 'CA1_CA2_CA3_Subiculum', 'DentatePyramids', 'Endothelial_Stalk', ..., 'Oligodendrocytes', 'Polydendrocytes', 'Subiculum_Entorhinal_cl2', 'Subiculum_Entorhinal_cl3']

In [7]:
import time
start_time = time.time()
# ################## Model training
sedr_net = SEDR_Train(adata_X, graph_dict, params)
if params.using_dec:
    sedr_net.train_with_dec()
else:
    sedr_net.train_without_dec()
sedr_feat, _, _, _ = sedr_net.process()

np.savez(f'{params.save_path}/SEDR_result.npz', sedr_feat=sedr_feat, deep_Dim=params.feat_hidden2)
end_time = time.time()

GNN model train without DEC:  |################| 200 / 200 | Left time: 0.01 mins| Loss: 22.3342
Training Graph Net with DEC loss:  |################| 200 / 200 | Loss: 22.1735


In [8]:
# ################## Result plot
adata_sedr = anndata.AnnData(sedr_feat)
adata_sedr.uns["Time"] = end_time-start_time
# adata_sedr.uns['spatial'] = adata_h5.uns['spatial'] #Slide_seqV2这个数据集没有uns里面的sptial文件夹
adata_sedr.obsm['spatial'] = adata_h5.obsm['spatial']

sc.pp.neighbors(adata_sedr, n_neighbors=params.eval_graph_n)
sc.tl.umap(adata_sedr)

eval_resolution = res_search_fixed_clus(adata_sedr, n_clusters)

sc.tl.leiden(adata_sedr, key_added="SEDR_leiden", resolution=eval_resolution)

### 没有img_key画不出图
# sc.pl.spatial(adata_sedr, img_key="hires", color=['SEDR_leiden'], show=False)
# plt.savefig(os.path.join(params.save_path, "SEDR_leiden_plot.pdf"), bbox_inches='tight', dpi=300)

2022-12-27 09:37:04.566952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 09:37:04.796844: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-27 09:37:04.853508: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-27 09:37:05.507326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [12]:
adata_sedr

AnnData object with n_obs × n_vars = 26146 × 28
    obs: 'leiden', 'SEDR_leiden'
    uns: 'Time', 'neighbors', 'umap', 'leiden'
    obsm: 'spatial', 'X_umap'
    obsp: 'distances', 'connectivities'

In [13]:
adata_sedr.obs["SEDR_leiden"]

0         4
1        12
2         2
3         2
4        10
         ..
26141     6
26142     0
26143    11
26144     6
26145     0
Name: SEDR_leiden, Length: 26146, dtype: category
Categories (14, object): ['0', '1', '2', '3', ..., '10', '11', '12', '13']

In [11]:
adata_h5.obs["cluster"]

TACTTTAGCGCAGT    CA1_CA2_CA3_Subiculum
TCGATATGGCACAA          DentatePyramids
ACGGATGTTCCGAT         Oligodendrocytes
TCTCATGGGTGGGA               Astrocytes
ACAGGGTTTATCGA    CA1_CA2_CA3_Subiculum
                          ...          
CTGGCTTTAGACCT    CA1_CA2_CA3_Subiculum
CGTCGGATGTAGGG                Microglia
CTTGATTAGGCAGG             Interneurons
GCGCTCAGTGCTCT             Interneurons
GAGATAACAGTTCC               Astrocytes
Name: cluster, Length: 26146, dtype: category
Categories (14, object): ['Astrocytes', 'CA1_CA2_CA3_Subiculum', 'DentatePyramids', 'Endothelial_Stalk', ..., 'Oligodendrocytes', 'Polydendrocytes', 'Subiculum_Entorhinal_cl2', 'Subiculum_Entorhinal_cl3']

In [19]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
ARI = adjusted_rand_score(adata_sedr.obs["SEDR_leiden"].to_list(), adata_h5.obs["cluster"].to_list()) ## ARI 是从 包sklearn.metrics.cluster中导入的衡量标准
print('Adjusted rand index = %.2f' %ARI)    
adata_sedr.uns["ARI"] = ARI

Adjusted rand index = 0.18


In [20]:
adata_sedr.uns["ARI"]

0.17503938621681597

In [21]:
import pickle
file = open("./Results/Slide-seqV2_crop_SEDR.pickle","wb")
pickle.dump(adata_sedr,file)
file.close()

In [ ]:
adata_sedr.obsm["spatial"] = adata_sedr.obsm["spatial"] * (-1)
sc.pl.embedding(adata_sedr, basis="spatial", color="SEDR_leiden",s=6, show=False, title='SEDR')
plt.axis('off')

sc.pl.umap(adata_sedr,color=["SEDR_leiden"],title='SEDR')

In [ ]:
adata_filename = "SEDR"+"_Slide_seqV2"+".h5ad"
adata_sedr.filename = './Results/'+adata_filename